In [830]:
from dotenv import load_dotenv
import os

load_dotenv()

if os.getenv("GROQ_API_KEY"):
    print('Groq API Key loaded successfully')
else:
    print('Groq API Key loading failed, please make sure the .env file exists and the spelling is correct')

if os.getenv("TAVILY_API_KEY"):
    print('Tavily API Key loaded successfully')
else:
    print('Tavily API Key loading failed, please make sure the .env file exists and the spelling is correct')

Groq API Key loaded successfully
Tavily API Key loaded successfully


In [831]:
# read file wordlist-eng.txt
wordlist = open("wordlist-eng.txt", "r")
words = wordlist.readlines()
wordlist.close()

# remove newline characters
words = [word.strip() for word in words]


print(words)

['AFRICA', 'AGENT', 'AIR', 'ALIEN', 'ALPS', 'AMAZON', 'AMBULANCE', 'AMERICA', 'ANGEL', 'ANTARCTICA', 'APPLE', 'ARM', 'ATLANTIS', 'AUSTRALIA', 'AZTEC', 'BACK', 'BALL', 'BAND', 'BANK', 'BAR', 'BARK', 'BAT', 'BATTERY', 'BEACH', 'BEAR', 'BEAT', 'BED', 'BEIJING', 'BELL', 'BELT', 'BERLIN', 'BERMUDA', 'BERRY', 'BILL', 'BLOCK', 'BOARD', 'BOLT', 'BOMB', 'BOND', 'BOOM', 'BOOT', 'BOTTLE', 'BOW', 'BOX', 'BRIDGE', 'BRUSH', 'BUCK', 'BUFFALO', 'BUG', 'BUGLE', 'BUTTON', 'CALF', 'CANADA', 'CAP', 'CAPITAL', 'CAR', 'CARD', 'CARROT', 'CASINO', 'CAST', 'CAT', 'CELL', 'CENTAUR', 'CENTER', 'CHAIR', 'CHANGE', 'CHARGE', 'CHECK', 'CHEST', 'CHICK', 'CHINA', 'CHOCOLATE', 'CHURCH', 'CIRCLE', 'CLIFF', 'CLOAK', 'CLUB', 'CODE', 'COLD', 'COMIC', 'COMPOUND', 'CONCERT', 'CONDUCTOR', 'CONTRACT', 'COOK', 'COPPER', 'COTTON', 'COURT', 'COVER', 'CRANE', 'CRASH', 'CRICKET', 'CROSS', 'CROWN', 'CYCLE', 'CZECH', 'DANCE', 'DATE', 'DAY', 'DEATH', 'DECK', 'DEGREE', 'DIAMOND', 'DICE', 'DINOSAUR', 'DISEASE', 'DOCTOR', 'DOG', 'DRAFT',

In [832]:
import re
import httpx
import os

from langchain_groq import ChatGroq
load_dotenv()

chat_model = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [833]:
import random


In [834]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_groq import ChatGroq

In [835]:
class SpymasterState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    clue: str
    n_words: int

class GuesserState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    clue: str
    guess_number: int
    guesses: Annotated[list[str], operator.add]
    last_guess_correct: bool

chat_model = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [836]:
class Board:
    def __init__(self,word_list,random_state=42):
        random.seed(random_state)
        self.random_state = random_state
        self.word_list = word_list

        self._prepare_grid()

    def _prepare_grid(self):
        words = random.sample(self.word_list, 25)

        n_words_team_1 = 9
        n_words_team_2 = 8
        n_words_bomb = 1
        chosen_words = random.sample(words, n_words_team_1 + n_words_team_2 + n_words_bomb)

        words_team_1 = chosen_words[:n_words_team_1]
        words_team_2 = chosen_words[n_words_team_1:n_words_team_1 + n_words_team_2]
        words_bomb = chosen_words[n_words_team_1 + n_words_team_2:]

        self.grid = {
            word: {
                "team":1 if word in words_team_1 else 2 if word in words_team_2 else "X" if word in words_bomb else 0,
                "revealed":False
                } 
            for word in words
            }
        self.grid_hidden = {word: {"team":"?","revealed":False} for word in words}

    def get_grid(self):
        return self.grid
    
    def get_hidden_grid(self):
        self._update_hidden_grid()
        return self.grid_hidden
        
    def reveal_word(self,word):
        self.grid[word]["revealed"] = True
        self._update_hidden_grid()
        return self.grid[word]["team"] 
    
    def _update_hidden_grid(self):
        for word in self.grid:
            if self.grid[word]["revealed"]:
                self.grid_hidden[word]["team"] = self.grid[word]["team"]
                self.grid_hidden[word]["revealed"] = True

In [837]:
class Spymaster:
    def __init__(self, model, prompt_func, grid: Board, team):
        self.model = model
        # prepare the StateGraph
        graph = StateGraph(SpymasterState)
        # create the nodes of the graph and corresponding behaviour
        graph.add_node("head", self.create_clue)
        graph.set_entry_point("head")
        # create the graph and turn it into a LangChain Runnable (allows to call methods like .invoke,...)
        self.graph = graph.compile()
        self.prompt_func = prompt_func
        self.team=team
        self.grid = grid

    # create a clue (all in one -> expand later on)
    def create_clue(self, state: SpymasterState):
        messages = [SystemMessage(content=self.prompt_func(self.team))] + [SystemMessage(str(self.grid.get_grid()))]
        message = self.model.invoke(messages)
        clue, n_words = self.extract_clue(message.content)
        print(message.content) # TODO: remove
        return {
            'messages': [message],
            'clue': clue,
            'n_words': n_words}
    
    def extract_clue(self, message):
        # extract clue and number of words from message using regex of the form "-[<clue>,<number>]-"
        pattern = r"-\[(.*),(\d+)\]-"
        match = re.search(pattern, message)
        print(message)
        if match:
            return match.group(1), int(match.group(2))
        else:
            return None, None

In [838]:
class Guesser:
    def __init__(self, model, system_prompt_planner_func, system_prompt_guesser_func, grid: Board, team, clue, n_words):
        self.model = model
        # set the system message
        self.system_prompt_planner = system_prompt_planner_func(team)
        self.system_prompt_guesser = system_prompt_guesser_func(team)
        # set the grid
        self.grid = grid
        # set the team
        self.team = team

        self.clue = clue
        self.n_words = n_words

        # prepare the StateGraph
        graph = StateGraph(GuesserState)
        # create the nodes of the graph and corresponding behaviour
        graph.add_node("planner", self.planner)
        graph.add_node("prepare_guess", self.prepare_guess)
        graph.add_edge("planner", "prepare_guess")
        graph.add_conditional_edges("prepare_guess", self.new_guess_available, {True: "prepare_guess", False: END})
        
        graph.set_entry_point("planner")
        # create the graph and turn it into a LangChain Runnable (allows to call methods like .invoke,...)
        self.graph = graph.compile()

    # plan the round
    def planner(self, state: GuesserState):
        new_info = f"""
        The clue is: {self.clue} with {self.n_words} words. The grid is: {self.grid.get_hidden_grid()}. Assign the a percentage of confidence to each word."""
        
        messages = [SystemMessage(content=self.system_prompt_planner)] + [SystemMessage(content=new_info)]
        message = self.model.invoke(messages)
        return {'messages': [SystemMessage(content=new_info)] + [message]}
    
    # create a guess (all in one -> expand later on)
    def prepare_guess(self, state: GuesserState):
        messages = [SystemMessage(content=self.system_prompt_guesser)] + state['messages']
        print(f"messages: {messages}") # TODO: remove
        message = self.model.invoke(messages)
        print(f"guess: {message.content}") # TODO: remove
        guess = self.extract_guess(message.content)

        try:
            gn = state['guesses'] + 1
        except:
            gn = 1

        return {
            'messages': [message],
            'guesses': [guess],
            'guess_number': gn}
    
    def take_guess(self, state: GuesserState):
        guess = state['guesses'][-1]
        print(f"taking guess {guess=}")
        team = self.grid.reveal_word(guess)
        return team == self.team # return True if the guess was correct, False otherwise
    
    def new_guess_available(self, state: GuesserState):
        if state['guesses'][-1] == "!END!":
            return False
        
        guess_correct = self.take_guess(state)
        if not guess_correct:
            return False
        if state['guess_number'] == self.n_words+1:
            return False
        return True

    def extract_guess(self, message):
        # extract guess from message using regex of the form "-[<guess>]-"; for ending the turn it should be "[guess: !END!]"
        pattern = r'-\[[A-Z]+\]-'
        print(f"{message=}")
        match = re.search(pattern, message)
        if match:
            #print(f"{match.group(1)=}")
            print(f"{match=}")
            return match.group(0)[2:-2]
        else:
            return None

In [839]:
def get_prompt_spymaster(team):
    prompt = f""" You take a role in a game and play for team {team}.
    You are given a list of words. For each word, you will see which team it belongs to and wether it was already revealed to your counterpart.
    Your task is to find words, that belong to your team, were not revealed yet and share some similiarities. You then need to find a word that connects these words. This is called clue. You also need to provide the number of words, you tried to connect with your clue. Give your answer in the format: -[<clue>,<number>]-.
    """
    return prompt

def get_prompt_guesser_planner(team):
    prompt = f"""You take a role in a game and play for team {team}.
    You are given a list of words. For each word, you will see, if it has already been revealed. You are also given a clue and a number. Using this clue assign probabilities to each word of the list, that has not yet been revealed, that the clue is related to it. 
    """
    return prompt

def get_prompt_guesser_prepareGuess(team):
    prompt = f"""You take a role in a game and play for team {team}.
    You are given a list of words. For each word, you will see, if it has already been revealed. You are also given a clue and a number. Another player has assigned probabilities to each word of the list, that has not yet been revealed, that the clue is related to it. You can guess one word at a time. If you you are correct, you can continue guessing. If you are wrong, your turn ends. 
    You can guess by providing a word in the format -[<guess>]-.
    If you are unsure, you can end your turn by guessing "!END!".
    """
    return prompt

In [840]:
class Match:
    def __init__(self,words):

        self.grid = Board(words)

        self.current_team = 1 # 1 or 2
        self.current_player = "spymaster" # spymaster or guesser

        winner = self.game_loop()
        print(f"Team {winner} wins!")


    def game_loop(self):
        while True:
            # turn of spymaster
            sp = Spymaster(chat_model, get_prompt_spymaster, self.grid, self.current_team)
            messages_sp = [SystemMessage(content=f"You are playing for team {self.current_team}. The grid is: {self.grid.get_grid()}")]
            result = sp.graph.invoke({"messages": messages_sp})
            clue = result['clue']
            n_words = result['n_words']
            print(f"Team {self.current_team} spymaster gives clue: {clue} for {n_words}")

            # turn of guesser
            gu = Guesser(chat_model, get_prompt_guesser_planner,get_prompt_guesser_prepareGuess, self.grid, self.current_team, clue, n_words)
            messages_gu = [SystemMessage(content=f"You are playing for team {self.current_team}. The grid is: {self.grid.get_hidden_grid()}. The clue is: {clue} for {n_words} words.")]
            result = gu.graph.invoke({"messages": messages_gu})
            print(f"Team {self.current_team} guesser guessed: {result['guesses']}")

            # check if game is over
            game_over = self.check_game_over(self.current_team)
            if game_over:
                return game_over # representing the winning team
            
            # switch players
            if self.current_player == "guesser":
                self.current_team = 1 if self.current_team == 2 else 2
            self.current_player = "guesser" if self.current_player == "spymaster" else "spymaster"
            
    def check_game_over(self, current_team):
        team_1_revealed = [self.grid.grid[word]["revealed"] for word in self.grid.grid if self.grid.grid[word]["team"] == 1]
        team_2_revealed = [self.grid.grid[word]["revealed"] for word in self.grid.grid if self.grid.grid[word]["team"] == 2]
        bomb_revealed = [self.grid.grid[word]["revealed"] for word in self.grid.grid if self.grid.grid[word]["team"] == "X"]

        if all(team_1_revealed):
            return 1
        elif all(team_2_revealed):
            return 2
        elif any(bomb_revealed):
            return not current_team # the other team wins if bomb was revealed
        else:
            return False

In [841]:
match = Match(words)

After analyzing the given list of words, I found some words that belong to my team (team 1), were not revealed yet, and share some similarities. 

The words I found are: MEXICO, AUSTRALIA, and BUFFALO. These words are all names of countries or places.

A clue that connects these words is: COUNTRY.

I tried to connect 3 words with my clue.

My answer is: -[COUNTRY,3]-
After analyzing the given list of words, I found some words that belong to my team (team 1), were not revealed yet, and share some similarities. 

The words I found are: MEXICO, AUSTRALIA, and BUFFALO. These words are all names of countries or places.

A clue that connects these words is: COUNTRY.

I tried to connect 3 words with my clue.

My answer is: -[COUNTRY,3]-
Team 1 spymaster gives clue: COUNTRY for 3
messages: [SystemMessage(content='You take a role in a game and play for team 1.\n    You are given a list of words. For each word, you will see, if it has already been revealed. You are also given a clue and a number

KeyError: None